## HNSW(Hierarchical Navigable Small World)
HNSW是通过图的方式来解决向量搜索问题的算法，由Y.Malkov与D.Yashunin在[论文](https://arxiv.org/pdf/1603.09320.pdf)中首次提出。

这一个Section安排如下
1. 图拥有什么样的性质可以有效的找到最近的K个向量
2. NSW(Navigable Small World)
3. HNSW(Hierarchical Navigable Small World) 

### 1. 图的性质
我们先直观的感受一下使用图的方式来表现向量空间。
<div style="text-align: center">
<img src="./resources/raw_vector.png"/>
</div>

图中的点代表向量，我们可以看到，如果两个向量的距离较近，那么在图中这两个点之间的距离也会更近。当我们想要通过图的方式来解决向量搜索时，我们会希望从任一点出发可以到达图中其他所有的点，即这个图是一张联通图。
<div style="text-align: center">
<img src="./resources/compare_vec.png"/>
</div>

但仅仅只是联通图，仍然无法做到快速有效的找到距离最近的K个向量。考虑如下的情况，A点与B点之间相隔较远，因此如果想要从A点到达B点需要途经许多点(代表着大量的计算)，同时我们可以看到点C与许多其他的点都有连接，因此如果我们从点C开始寻找距离查询向量最近的K个点，我们会计算大量无关的点(因为与点C相连的点，其中很多大概率是与结果无关的)。
<div style="text-align: center">
<img src="./resources/two_conn.png"/>
</div>

综上所述，为了可以高效而准确的找到距离查询向量最近的K个向量。我们希望构建的图有以下几个性质
1. 联通图(没有孤岛)
2. 距离较远的点,有边可以相连(long range edge)
3. 构建的图中边的数量不宜过多(大量的计算)
4. 距离相近的点，有边连接（保证召回率）
其中3,4是召回率与计算量的tradeoff。





### 2. NSW
NSW通过有效且简单的算法构建出满足上述要求的图，下面分别从构建以及查询两个方面来介绍NSW算法。

#### 1. 构建
首先我们将通过随机的方式，将向量一个一个添加到图中，每一个新添加的点都会与当前图中距离该点最近的**M**个点相连。之所以通过**M**对相连的点的数量进行限制，是为了防止连接的边过多，从而影响查询效率。

我们通过一个例子来描述构建的过程，假设我们将**M**设置为3，并且已经将待加入的向量随机打乱。

首先，添加点A，因为当前图中没有其他任何的点，所以我们只需要添加A，而不用作任何其他的操作。后面我们继续添加点B，此时图中只有点A，点的个数小于3,因此我们可以直接将两者相连。

<div style="text-align: center">
<img src="./resources/addB.png"/>
</div>

类似的我们向图中加入点C，点D，我们会获得以下的图


<div style="text-align: center">
<img src="./resources/addCD.png"/>
</div>

随后我们继续添加点E，此时我们会找到当前图中距离点E最近的**M**个点，即A，B，C并将其相互连接。

<div style="text-align: center">
<img src="./resources/addE.png"/>
</div>

用相同的方式，我们继续添加点F，G，H，最终得到的图如下所示。

我们逐个检查NSW图是否满足我们之前要求的性质
1. 联通图，显而易见这是一张联通图
2. 距离较远的点有边可以相连，我们可以发现因为随机添加，最开始认为距离较近的点，比如A，D，随着添加的点越来越多，A，D相连的这条边成为了一条long range边。
3. 构建的图中，边的数量不宜过多。这一条因为我们始终用**M**控制边的数量，所以也可以满足
4. 距离较近的点，有边连接。因为我们始终与距离最近的**M**条边相连，因此也满足了该要求。

<div style="text-align: center">
<img src="./resources/addFH.png"/>
</div>

因此，我们只需要随机添加向量，并且在随机添加的过程中，与当前最近的**M**个点相连，我们就可以构建出一幅可以高效的进行ANN查询的图。下面我们讨论搜索的过程。

#### 2. 搜索

因为我们通过NSW构建出的图,具有良好的特性,因此我们只需要使用简单的贪心算法就可以获得较好的搜索结果。在给定了一个query point后
1. 我们在图中随机的选择一个点作为出发点(entry point)
2. 我们计算每一个与该点相连的点，选出最近的一个点。
    
    a. 若该点即为entry point,搜索结束,返回entry point。
    
    b. 若该点不为entry point, 设置该点为entry point, 重复过程2

下图为搜索的示意图,我们可以看到因为有long range，这一高速通道的存在，我们可以快速搜索到结果。

<div style="text-align: center">
<img src="./resources/nsw-search.png"/>
</div>



### 3.HNSW
尽管NSW已经可以很好的为我们解决ANN查询的问题，但其仍然有不足之处。
1. 搜索时，NSW无法区分long range与short range，从而无法先查询long range再查询short range。
2. 当数据的聚类效应特别明显时，即使我们乱序加入向量，cluster之间相互连接的边仍然十分稀疏，从而搜索结果容易陷入局部最优，同时效率也会比较低下。

因此为了解决上述问题，HNSW作为NSW的改良版被提了出来。

#### 1.构建
我们首先直观的感受HNSW图。我们可以看到hnsw相比于nsw多了层级的概念。我们从图中可以看到，level0中有全部的向量，随着层数的增加，向量的数量也相应的减少。
<div style="text-align: center">
<img src="./resources/hnsw.png"/>
</div>

HNSW并不要求我们乱序插入向量，当我们向HNSW添加新的向量时，我们首先会通过一个指数衰减的概率函数，得到这个向量所处的最大层级(如果最大层级计算出来是3,那么level3, level2,level1,level0中都含有这个向量)。

这就意味着，绝大多数的向量所处的最高层级都是level0, 同样我们也可以认为高层级是低层级的草图（抽样），因此高层级中的向量之间大概率是long range连接，低层级中的向量则是short range连接。这样子做给我们带来的好处就是搜索时，我们可以先寻找long range的边，再寻找short range的边，即先粗查再精查。从而尽可能减少搜索的次数。

当得到这个向量所处的最大层级后，我们便需要将其添加到图中。假设，新增的向量为**V**, 这个向量所处的最大层级为**I**,HNSW的最高层级为**J**。添加向量时因为需要从最高层级**J**，一直走到最低层级0,我们将添加时所处的层级设置为**C**。添加的过程可以分为3个阶段

1. **J** >= **C** > **I**
    这一阶段我们使用NSW的贪心算法，寻找距离最近的向量，随后在下一层级以这个点为搜索起点.
2. **I** >= **C** > 0
    在这一阶段，我们不仅要找距离最近的向量，我们还需要将**V**存放到这一级的图中。我们仍旧使用贪心算法寻找距离最近的向量，不同之处在于我们会维护一个动态的列表，保存距离**V**最近的`efCounstruction`个向量，`efConstruction`为可调节的参数。当我们在这一层搜索完成后，我们会将这一动态列表作为`Candidate`,并从中取出`M`个向量，与其连接。
3. **I** = 0
    这一阶段，我们采用和第二阶段一样的策略，不同的是在level0,向量**V**可以与最多**2M**个向量进行连接。

下图为一个简单的示例，新增向量所处的最高层级为1。我们首先在level2中，寻找与其最近的点(黄色标示)，找到后，我们以这个点为起点在level1中寻找与其最近的`efCounstruction`个点，随后于其中的**M**个向量进行连接。最后当我们到达level0时，我们用上一层连接的**M**个向量作为起点寻找符合要求的**2M**个向量，并于其相连。
<div style="text-align: center">
<img src="./resources/new-insert.png"/>
</div>





In [ ]:
import numpy as np
class HNSW:

    def __init__(self, dim, M, efConstruction) -> None:
        """
        Initialize the HNSW object.

        Args:
            dim (int): Dimension of the data.
            M (int): Number of edges per node.
            efConstruction (int): Parameter for controlling the maximum size of the dynamic list.

        Returns:
            None
        """
        self.datas = {}
        self.enterPoint = -1
        self.maxLevel = -1
        self.dim = dim
        self.efConstruction = efConstruction
        self.edges = {}
        self.Mmax = M
        self.Mmax0 = 2 * M
        self.levelMult = 1 / np.log(1.0 * M)
        self.queryIndexes = 100000

    def add(self, point: np.ndarray, label: int) -> None:
        """
        add() adds a point to the HNSW graph.

        Args:
            point (np.array): The point to be added to the graph.
            label (str): The label associated with the point.

        Returns:
            None
        """
        pass 

    def search(self, query: np.ndarray, efSearch: int) -> list:
        """
        search() searches the HNSW graph for the nearest neighbours of a query point.

        Args:
            query (np.array): The query point.
            efSearch (int): The efSearch parameter.

        Returns:
            list: The list of nearest neighbours.
        """
        pass
    def __distance(self, label1: int, label2: int) -> float:
        """
        Calculates the Euclidean distance between two points.

        Args:
            label1 (int): The first point.
            label2 (int): The second point.

        Returns:
            float: The Euclidean distance between x and y.
        """
        return np.linalg.norm(self.datas[label1] - self.datas[label2])

    def __generateLevel(self) -> int:
        """
        Generates a random level for a new node.

        Returns:
            int: The random level.
        """
        distribution = np.random.uniform(0.0, 1.0)
        r = -np.log(distribution) * self.levelMult
        return int(r)

    def __search_layer(self, label: int, eps: set, layer: int, efSearch: int) -> PriorityQueue:
        """
        Searches a single layer of the graph.

        Args:
            label (int): The label of the node to search.
            ep    (int): The label of the enter points.
            layer (int): The layer to search.
            efSearch (int): The efSearch parameter.

        Returns:
            PriorityQueue: The set of currently found nearest neighbours.
        """
        pass 

    def __select_neighbors(self, label: int, candidates: PriorityQueue, M: int) -> set:
        """
        Selects the M nearest neighbours of a node.

        Args:
            label (int): The label of the node.
            candidates (list): The list of candidates.
            M (int): The number of neighbours to select.

        Retruns:
            result(set): The set of M nearest neighbours.
        """
        pass